In [108]:
import re

def armenian_text(text):
    pattern_brackets = r"\((.*?)\)"
    clean_text = re.sub(pattern_brackets, "", text)
    arm_range = set(range(ord('Ա'), ord('Ֆ') + 1)).union(set(range(ord('ա'), ord('և') + 1)))
    clean_text = re.sub(r'[^\w\s]|_', '', re.sub(r'\d+', '', clean_text))
    accept = True
    for item in clean_text.split():
        if any([ord(char) not in arm_range for char in item]):
            accept = False
            break
    if accept:
        return text

In [109]:
allowed_punctuations = {',', '։', '՝', '՞', '-', '.'}
global allowed_punctuations
arm_range = set(range(ord('Ա'), ord('Ֆ') + 1)).union(set(range(ord('ա'), ord('և') + 1)))
global arm_range

In [110]:
import pickle
    
good_sentences = set()
good_sentences_with_latin = set()
with open("corpus26shamshyan.pkl", "rb") as f:  # CB_DATA
    data1 = pickle.load(f)
    data1 = [part for string in data1 for part in string.split('։')]
    
    for line in data1:
        # print(line)
        raw_text = armenian_text(line)
        # print(raw_text)
        if raw_text:
            good_sentences.add(raw_text)
        else:
            good_sentences_with_latin.add(line)

In [111]:
len(good_sentences)

13

In [112]:
def clean_sentence(sentence):
    new_sentence = ''
    for char in sentence:
        if char in allowed_punctuations or ord(char) in arm_range or char.isdigit():
            new_sentence += char
        elif char == "՛":
            new_sentence +=''
        else:
            new_sentence += ' '
    new_sentence = " ".join(new_sentence.split()).strip()
    return new_sentence

def clean_sentences(sentences):
    new_sentences = set()
    for sentence in sentences:
        new_sentence = clean_sentence(sentence)
        new_sentences.add(new_sentence)
    new_sentences = set([sentence for sentence in new_sentences if len(sentence.split()) >= 6])
    return new_sentences

new_sentences = clean_sentences(sentences=good_sentences)


In [113]:
def remove_repeating(sentence: str, allowed_punctuations: set) -> str:
    new_sentence = []
    for char in sentence:
        if char in allowed_punctuations and char != '՞':
            new_sentence.append(' ' + char + ' ')
        elif char.isalpha() or char.isdigit() or char == ' ' or char == '՞':
            new_sentence.append(char)
    final_sentence = []
    for item in ''.join(new_sentence).split():
        if item.isalpha() or item.isdigit():
            final_sentence.append(item)
        else:
            if len(final_sentence) > 0 and item != final_sentence[-1]:
                final_sentence.append(item)
    final_sentence = [word.replace('ւ','վ') if word.startswith('ւ') else word for word in final_sentence]
    return ' '.join(final_sentence).replace('եւ', 'և')

In [114]:
final_sentences_cc100 = [remove_repeating(sentence=sentence, allowed_punctuations=allowed_punctuations) \
                        for sentence in new_sentences]

In [115]:
len(final_sentences_cc100)

13

In [116]:
def improve(sentence):
    improved = []
    for item in sentence.split():
        if item == "եվ":
            improved.append("և")
        elif item == ":":
            improved.append("։")
        else:
            improved.append(item)

    if improved:
        if not improved[-1].isalpha():
            improved[-1] = "։"
        else:
            improved.append("։")
    else:
        # If sentence is empty, add an Armenian punctuation mark to avoid an empty result
        improved.append("։")

    return ' '.join(improved)

    
def final_improvement(final_sentences):
    final_sentences_improved = []
    for sentence in final_sentences:
        improved = improve(sentence)
        final_sentences_improved.append(improved)
    return final_sentences_improved

In [117]:
final_sentences_cc100_improved = final_improvement(final_sentences_cc100)

In [118]:
allowed_punctuations

{',', '-', '.', '՝', '՞', '։'}

In [119]:
def remove_space_before_punctuation(sentence):
    new_sent = []
    for idx, char in enumerate(sentence):
        if char in allowed_punctuations and idx > 0 and sentence[idx-1] == ' ':
            new_sent.pop()
            new_sent.append(char)
        else:
            new_sent.append(char)
    return ''.join(new_sent)

In [120]:
final_sentences_cc100 = []
for sentence in final_sentences_cc100_improved:
    final_sentences_cc100.append(remove_space_before_punctuation(sentence=sentence))


In [123]:
final_sentences_cc100

['Մերկասառույցի պայմաններում վարորդներին չհաջողվեց դուրս բերել ձորակում հայտնված բեռնատարը Ներկայացնում ենք տեսանյութը։',
 'Թաքնված տեսախցիկը կատակում է թանգարանի այցելուների հետ Ներկայացնում ենք տեսանյութը։',
 'Համացանցում հայտնված տեսանյութում տղամարդը ցույց է տալիս, թե ինչպես է կարելի գորտ ուտել Չնայած այն բանին, որ որոշ երկրներում, գորտը համարվում է, դելիկատես, սակայն նրա առաջարկած տարբերակը յուրօրինակ է։',
 'Թուրքական հեռուստաալիքի հաղորդավարը եթերի ժամանակ միկրոֆոնի փոխարեն շփոթմամբ մեկ այլ առարկա է օգտագործում Ներկայացնում ենք տեսանյութը։',
 'Հայտնի դերասան Վին Դիզելը համացանցում տեղադրել է տեսանյութ, որտեղ պարում է Բեյոնսեի և Քեթի Պերի երգի ներքո Նա ապացուցել է, որ ինքը ոչ միայն լավ դերասան է, այլ նաև լավ պարող է։',
 'Թե ինչու են կանայք ավելի երկար ապրում, դիտեք ֆոտոշարքում։',
 'Ռուսաստանի գյուղերից մեկում ձուկ որսալը, պարզվում է, այդքան էլ դժվար չէ Ներկայացնում ենք տեսանյութը։',
 'ԱՄՆ- ի ավտոճանապարհներից մեկում հաջողակ վարորդը հրաշքով էր խուսափել հանդիպակաց գոտում հայտնված բե

In [124]:
import pickle

with open('thired.pkl', 'wb') as handle:
    pickle.dump(final_sentences_cc100, handle, protocol=pickle.HIGHEST_PROTOCOL)